<a href="https://colab.research.google.com/github/Rt247/Not_NLP_CW/blob/BERT_method/sentence_level_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Download datasets:

In [0]:
from os.path import exists

if not exists('enzh_data.zip'):
    !wget -O enzh_data.zip https://competitions.codalab.org/my/datasets/download/03e23bd7-8084-4542-997b-6a1ca6dd8a5f
    !unzip enzh_data.zip

Check data downloaded successfully:

In [0]:
with open("./train.enzh.src", "r") as enzh_src:
  print("Source: ",enzh_src.readline())
with open("./train.enzh.mt", "r") as enzh_mt:
  print("Translation: ",enzh_mt.readline())
with open("./train.enzh.scores", "r") as enzh_scores:
  print("Score: ",enzh_scores.readline())

### English Models Setup

Download English models:

In [0]:
!spacy download en_core_web_md
!spacy link en_core_web_md en300

Load a GloVe English model with dim 100.

Some Chinese models only have **dim 100**, so we will need to **tokenize with spaCy, then embed with GloVe**.

In [0]:
import torchtext
import spacy

# Embedding for English when dim 100
glove = torchtext.vocab.GloVe(name='6B', dim=100)

# Tokenizer for English when dim 100, Tokenizer and Embedding when dim 300
nlp_en = spacy.load('en300')


Functions for processing English dataset:

In [0]:
import numpy as np
import torch
from nltk import download
from nltk.corpus import stopwords

#downloading stopwords from the nltk package
download('stopwords') #stopwords dictionary, run once
stop_words_en = set(stopwords.words('english'))


def preprocess_en(sentence, nlp):
    text = sentence.lower()
    doc = [token.lemma_ for token in  nlp.tokenizer(text)]
    doc = [word for word in doc if word not in stop_words_en]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

def get_word_vector_en(embeddings, word):
    try:
      vec = embeddings.vectors[embeddings.stoi[word]]
      return vec
    except KeyError:
      #print(f"Word {word} does not exist")
      pass
      

def get_sentence_emb_en(line, nlp):
  text = line.lower()
  l = [token.lemma_ for token in nlp.tokenizer(text)]
  l = ' '.join([word for word in l if word not in stop_words_en])

  sen = nlp(l)
  return sen.vector


### Chinese Models Setup

Download Chinese stopwords:

In [0]:
!wget -c https://github.com/Tony607/Chinese_sentiment_analysis/blob/master/data/chinese_stop_words.txt

Download and load Chinese model with **dim 100** (University of Oslo):

In [0]:
if not exists("zh_100.zip"):
  !wget -O zh_100.zip http://vectors.nlpl.eu/repository/20/35.zip
  !unzip zh_100.zip -d ./zh_100

from gensim.models import KeyedVectors

wv_from_bin_100 = KeyedVectors.load_word2vec_format("./zh_100/model.bin", binary=True) 

Functions for processing Chinese dataset:

In [0]:
import string
import jieba
import gensim 
import spacy
import numpy as np

stop_words = [ line.rstrip() for line in open('./chinese_stop_words.txt',"r", encoding="utf-8") ]

def processing_zh(sentence):
  seg_list = jieba.lcut(sentence,cut_all=True)
  doc = [word for word in seg_list if word not in stop_words]
  docs = [e for e in doc if e.isalnum()]
  return docs


### BERT embedding Setup


In [0]:
!pip install transformers
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, AdamW, BertConfig

In [0]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [0]:
#Scores
f_train_scores = open("./train.enzh.scores", 'r')
zh_train_scores = f_train_scores.readlines()
labels = np.array(zh_train_scores).astype(float)

In [0]:
model = BertModel.from_pretrained("bert-base-multilingual-cased")
model.cuda()

In [0]:
def token_raw_inputs(original, translated):
  # Load pre-trained model tokenizer (vocabulary)
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

  text_pairs = list(zip(original, translated))
  MAX_LENGTH = 128

  inputs = [tokenizer.encode_plus(original, text_pair=translated, add_special_tokens = True, max_length=MAX_LENGTH, pad_to_max_length=True) for original, translated in text_pairs]

  return [d['input_ids'] for d in inputs], [d['attention_mask'] for d in inputs]


def get_BERT_embedding(input_tokens, attention_masks):
  input_tensors = torch.tensor(input_tokens).to(device)
  attention_mask_tensors = torch.tensor(attention_masks).to(device)

  with torch.no_grad():
    last_hidden_states = model(input_tensors, attention_mask=attention_mask_tensors)
  return last_hidden_states[0][:,0,:].cpu().numpy()

## Process Scores

In [0]:
import spacy
import torchtext
from torchtext import data

f_train_scores = open("./train.enzh.scores", 'r')
zh_train_scores = f_train_scores.readlines()

f_val_scores = open("./dev.enzh.scores", 'r')
zh_val_scores = f_val_scores.readlines()

train_scores = np.array(zh_train_scores).astype(float)
y_train_zh = train_scores

val_scores = np.array(zh_val_scores).astype(float)
y_val_zh = val_scores

## Support Vector Machines


In [0]:
# Setup
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

from sklearn.svm import SVR
from scipy.stats.stats import pearsonr


### Using Average Word Embedding Vectors



In [0]:
def get_avg_sentence_vector_zh(line, word_vectors):
  vectors = []
  for w in line:
    try:
      emb = word_vectors[w]
      vectors.append(emb)
    except:
      pass #Do not add if the word is out of vocabulary
  if vectors:
    return np.mean(vectors, axis=0).tolist()
  return [0] * 100

def get_avg_embeddings_zh(f, word_vectors):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]
  for l in lines:
    sent  = processing_zh(l)
    vec = get_avg_sentence_vector_zh(sent, word_vectors)

    if vec is not None:
      sentences_vectors.append(vec)
    else:
      print(l)
  return sentences_vectors


def get_avg_sentence_vector_en(embeddings, line):
  vectors = []
  for w in line:
    emb = get_word_vector_en(embeddings, w)
    #do not add if the word is out of vocabulary
    if emb is not None:
      vectors.append(emb)
  if vectors:
    return torch.mean(torch.stack(vectors), dim=0).tolist()
  return [0] * 100

# assume dim 100
def get_avg_embeddings_en(f, embeddings, nlp):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]

  for l in lines:
    sentence = preprocess_en(l, nlp)
    try:
      vec = get_avg_sentence_vector_en(embeddings, sentence)
      sentences_vectors.append(vec)
    except:
      sentences_vectors.append(0)

  return sentences_vectors

In [0]:
zh_train_mt_100_a = get_avg_embeddings_zh("./train.enzh.mt", wv_from_bin_100)
zh_train_src_100_a = get_avg_embeddings_en("./train.enzh.src", glove, nlp_en)

zh_val_mt_100_a = get_avg_embeddings_zh("./dev.enzh.mt", wv_from_bin_100)
zh_val_src_100_a = get_avg_embeddings_en("./dev.enzh.src", glove, nlp_en)

In [0]:
X_train_100_a = [x + y for x, y in zip(zh_train_src_100_a, zh_train_mt_100_a)]
X_train_zh_100_a = np.array(X_train_100_a)

X_val_100_a = [x + y for x, y in zip(zh_val_src_100_a, zh_val_mt_100_a)]
X_val_zh_100_a = np.array(X_val_100_a)


In [0]:
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_a, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_a)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

"""
linear
RMSE: 0.9044962563186333 Pearson 0.3017781690203462

poly
RMSE: 0.8990697909416231 Pearson 0.3032902746054339

rbf
RMSE: 0.8900985622788053 Pearson 0.3403404558003603

sigmoid
RMSE: 7.152607007355879 Pearson -0.03977439348067312
"""

### Using Sum of Word Embedding Vectors

In [0]:
def get_sum_sentence_vector_zh(line, word_vectors):
  vectors = []
  for w in line:
    try:
      emb = word_vectors[w]
      vectors.append(emb)
    except:
      pass #Do not add if the word is out of vocabulary
  if vectors:
    return np.sum(vectors, axis=0).tolist()
  return [0] * 100

def get_sum_embeddings_zh(f, word_vectors):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]
  for l in lines:
    sent  = processing_zh(l)
    vec = get_sum_sentence_vector_zh(sent, word_vectors)

    if vec is not None:
      sentences_vectors.append(vec)
    else:
      print(l)
  return sentences_vectors


def get_sum_sentence_vector_en(embeddings, line):
  vectors = []
  for w in line:
    emb = get_word_vector_en(embeddings, w)
    #do not add if the word is out of vocabulary
    if emb is not None:
      vectors.append(emb)
  if vectors:
    return torch.sum(torch.stack(vectors), dim=0).tolist()
  return [0] * 100

# assume dim 100
def get_sum_embeddings_en(f, embeddings, nlp):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]

  for l in lines:
    sentence = preprocess_en(l, nlp)
    try:
      vec = get_sum_sentence_vector_en(embeddings, sentence)
      sentences_vectors.append(vec)
    except:
      sentences_vectors.append(0)

  return sentences_vectors

In [0]:
zh_train_mt_100_s = get_sum_embeddings_zh("./train.enzh.mt", wv_from_bin_100)
zh_train_src_100_s = get_sum_embeddings_en("./train.enzh.src", glove, nlp_en)

zh_val_mt_100_s = get_sum_embeddings_zh("./dev.enzh.mt", wv_from_bin_100)
zh_val_src_100_s = get_sum_embeddings_en("./dev.enzh.src", glove, nlp_en)

In [0]:
X_train_100_s = [x + y for x, y in zip(zh_train_src_100_s, zh_train_mt_100_s)]
X_train_zh_100_s = np.array(X_train_100_s)

X_val_100_s = [x + y for x, y in zip(zh_val_src_100_s, zh_val_mt_100_s)]
X_val_zh_100_s = np.array(X_val_100_s)

In [0]:
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_s, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_s)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

"""
linear
RMSE: 0.9203004562821944 Pearson 0.2525785904226626

poly
RMSE: 0.9499024011434094 Pearson 0.18792023737965777

rbf
RMSE: 0.905470160037738 Pearson 0.29378234780721474

sigmoid
RMSE: 34.73144893811673 Pearson -0.004944951711832229
"""

### Using Min/Max of Word Embedding Vectors

In [0]:
def get_min_sentence_vector_zh(line, word_vectors):
  vectors = []
  for w in line:
    try:
      emb = word_vectors[w]
      vectors.append(emb)
    except:
      pass #Do not add if the word is out of vocabulary
  if vectors:
    return np.amin(vectors, axis=0).tolist()
  return [0] * 100

def get_min_embeddings_zh(f, word_vectors):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]
  for l in lines:
    sent  = processing_zh(l)
    vec = get_min_sentence_vector_zh(sent, word_vectors)

    if vec is not None:
      sentences_vectors.append(vec)
    else:
      print(l)
  return sentences_vectors


def get_min_sentence_vector_en(embeddings, line):
  vectors = []
  for w in line:
    emb = get_word_vector_en(embeddings, w)
    #do not add if the word is out of vocabulary
    if emb is not None:
      vectors.append(emb)
  if vectors:
    return torch.min(torch.stack(vectors), dim=0)[0].tolist()
  return [0] * 100

# assume dim 100
def get_min_embeddings_en(f, embeddings, nlp):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]

  for l in lines:
    sentence = preprocess_en(l, nlp)
    try:
      vec = get_min_sentence_vector_en(embeddings, sentence)
      sentences_vectors.append(vec)
    except:
      sentences_vectors.append(0)

  return sentences_vectors


def get_max_sentence_vector_zh(line, word_vectors):
  vectors = []
  for w in line:
    try:
      emb = word_vectors[w]
      vectors.append(emb)
    except:
      pass #Do not add if the word is out of vocabulary
  if vectors:
    return np.amax(vectors, axis=0).tolist()
  return [0] * 100

def get_max_embeddings_zh(f, word_vectors):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]
  for l in lines:
    sent  = processing_zh(l)
    vec = get_max_sentence_vector_zh(sent, word_vectors)

    if vec is not None:
      sentences_vectors.append(vec)
    else:
      print(l)
  return sentences_vectors


def get_max_sentence_vector_en(embeddings, line):
  vectors = []
  for w in line:
    emb = get_word_vector_en(embeddings, w)
    #do not add if the word is out of vocabulary
    if emb is not None:
      vectors.append(emb)
  if vectors:
    return torch.max(torch.stack(vectors), dim=0)[0].tolist()
  return [0] * 100

# assume dim 100
def get_max_embeddings_en(f, embeddings, nlp):
  file = open(f) 
  lines = file.readlines() 
  sentences_vectors =[]

  for l in lines:
    sentence = preprocess_en(l, nlp)
    try:
      vec = get_max_sentence_vector_en(embeddings, sentence)
      sentences_vectors.append(vec)
    except:
      sentences_vectors.append(0)

  return sentences_vectors

In [0]:
zh_train_mt_100_min = get_min_embeddings_zh("./train.enzh.mt", wv_from_bin_100)
zh_train_src_100_min = get_min_embeddings_en("./train.enzh.src", glove, nlp_en)
zh_val_mt_100_min = get_min_embeddings_zh("./dev.enzh.mt", wv_from_bin_100)
zh_val_src_100_min = get_min_embeddings_en("./dev.enzh.src", glove, nlp_en)

zh_train_mt_100_max = get_max_embeddings_zh("./train.enzh.mt", wv_from_bin_100)
zh_train_src_100_max = get_max_embeddings_en("./train.enzh.src", glove, nlp_en)
zh_val_mt_100_max = get_max_embeddings_zh("./dev.enzh.mt", wv_from_bin_100)
zh_val_src_100_max = get_max_embeddings_en("./dev.enzh.src", glove, nlp_en)

In [0]:
X_train_100_min = [x + y for x, y in zip(zh_train_src_100_min, zh_train_mt_100_min)]
X_train_zh_100_min = np.array(X_train_100_min)
X_val_100_min = [x + y for x, y in zip(zh_val_src_100_min, zh_val_mt_100_min)]
X_val_zh_100_min = np.array(X_val_100_min)

X_train_100_max = [x + y for x, y in zip(zh_train_src_100_max, zh_train_mt_100_max)]
X_train_zh_100_max = np.array(X_train_100_max)
X_val_100_max = [x + y for x, y in zip(zh_val_src_100_max, zh_val_mt_100_max)]
X_val_zh_100_max = np.array(X_val_100_max)

In [0]:
print("min")
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_min, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_min)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

print("max")
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_max, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_max)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

"""
min
linear
RMSE: 0.9174155670157896 Pearson 0.26925288419123733

poly
RMSE: 0.9721279726125316 Pearson 0.23475167649201809

rbf
RMSE: 0.9049414520907609 Pearson 0.2920329610610498

sigmoid
RMSE: 26.865545536417198 Pearson 0.0076818897644271664

max
linear
RMSE: 0.9288259124772581 Pearson 0.2342204449314311

poly
RMSE: 1.0188986421056523 Pearson 0.18458085384794573

rbf
RMSE: 0.9074016100647716 Pearson 0.28092258260753544

sigmoid
RMSE: 26.529780761913454 Pearson -0.007926148030142141
"""


### Combinations

In [0]:
# min + max
X_train_100_mm = [sum(t, []) for t in zip(zh_train_src_100_min, zh_train_src_100_max, zh_train_mt_100_min, zh_train_mt_100_max)]
X_train_zh_100_mm = np.array(X_train_100_mm)
X_val_100_mm = [sum(t, []) for t in zip(zh_val_src_100_min, zh_val_src_100_max, zh_val_mt_100_min, zh_val_mt_100_max)]
X_val_zh_100_mm = np.array(X_val_100_mm)

# min + avg + max
X_train_100_mam = [sum(t, []) for t in zip(zh_train_src_100_min, zh_train_src_100_a, zh_train_src_100_max, zh_train_mt_100_min, zh_train_mt_100_a, zh_train_mt_100_max)]
X_train_zh_100_mam = np.array(X_train_100_mam)
X_val_100_mam = [sum(t, []) for t in zip(zh_val_src_100_min, zh_val_src_100_a, zh_val_src_100_max, zh_val_mt_100_min, zh_val_mt_100_a, zh_val_mt_100_max)]
X_val_zh_100_mam = np.array(X_val_100_mam)

# avg + sum
X_train_100_as = [sum(t, []) for t in zip(zh_train_src_100_a, zh_train_src_100_s, zh_train_mt_100_a, zh_train_mt_100_s)]
X_train_zh_100_as = np.array(X_train_100_mam)
X_val_100_as = [sum(t, []) for t in zip(zh_val_src_100_a, zh_val_src_100_s, zh_val_mt_100_a, zh_val_mt_100_s)]
X_val_zh_100_as = np.array(X_val_100_mam)

# min + avg + max + sum
X_train_100_mams = [sum(t, []) for t in zip(zh_train_src_100_min, zh_train_src_100_a, zh_train_src_100_max, zh_train_src_100_s, zh_train_mt_100_min, zh_train_mt_100_a, zh_train_mt_100_max, zh_train_src_100_s)]
X_train_zh_100_mams = np.array(X_train_100_mam)
X_val_100_mams = [sum(t, []) for t in zip(zh_val_src_100_min, zh_val_src_100_a, zh_val_src_100_max, zh_val_src_100_s, zh_val_mt_100_min, zh_val_mt_100_a, zh_val_mt_100_max, zh_val_mt_100_s)]
X_val_zh_100_mams = np.array(X_val_100_mam)

In [0]:
print("min + max")
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_mm, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_mm)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

print("min + avg + max")
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_mam, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_mam)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

print("avg + sum")
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_as, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_as)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

print("min + avg + max + sum")
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(X_train_zh_100_mams, y_train_zh)
    print(k)
    predictions = clf_t.predict(X_val_zh_100_mams)
    pearson = pearsonr(y_val_zh, predictions)
    print(f'RMSE: {rmse(predictions,y_val_zh)} Pearson {pearson[0]}')
    print()

"""
min + max
linear
RMSE: 0.9165262212864648 Pearson 0.26668369681069803

poly
RMSE: 0.9081786905379852 Pearson 0.3058041114768354

rbf
RMSE: 0.9084429289156349 Pearson 0.30464644770878846

sigmoid
RMSE: 3.5248833190245237 Pearson -0.006792767476776504

min + avg + max
linear
RMSE: 0.9081244309157551 Pearson 0.28851234087420935

poly
RMSE: 0.8982943971746072 Pearson 0.3296975280939544

rbf
RMSE: 0.8992479975159785 Pearson 0.3292233306043897

sigmoid
RMSE: 2.600000956750181 Pearson 0.014142349847365135

avg + sum
linear
RMSE: 0.9081244309157551 Pearson 0.28851234087420935

poly
RMSE: 0.8982943971746072 Pearson 0.3296975280939544

rbf
RMSE: 0.8992479975159785 Pearson 0.3292233306043897

sigmoid
RMSE: 2.600000956750181 Pearson 0.014142349847365135

min + avg + max + sum
linear
RMSE: 0.9081244309157551 Pearson 0.28851234087420935

poly
RMSE: 0.8982943971746072 Pearson 0.3296975280939544

rbf
RMSE: 0.8992479975159785 Pearson 0.3292233306043897

sigmoid
RMSE: 2.600000956750181 Pearson 0.014142349847365135
"""

# BERT embedding with regression model


## Feature extraction using BERT

Use BERT to generate sentence level embedding

In [0]:
from sklearn.linear_model import Ridge

# Tokenize our sentence with the BERT tokenizer.
original_texts = open("./train.enzh.src").readlines()
translated_texts = open("./train.enzh.mt").readlines()
input_ids, input_attention_masks = token_raw_inputs(original_texts, translated_texts)

train_features = get_BERT_embedding(input_ids[0:1000], input_attention_masks[0:1000])
for i in range(1, 7):
  features = get_BERT_embedding(input_ids[i*1000:(i + 1)*1000], input_attention_masks[i*1000:(i + 1)*1000])
  train_features = np.concatenate((train_features, features))
train_labels = labels

# Tokenize our sentence with the BERT tokenizer.
dev_original_texts = open("./dev.enzh.src").readlines()
dev_translated_texts = open("./dev.enzh.mt").readlines()

zh_val_scores = open("./dev.enzh.scores", 'r').readlines()
test_labels = np.array(zh_val_scores).astype(float)

test_input_ids, test_attention_masks = token_raw_inputs(dev_original_texts, dev_translated_texts)
test_features = get_BERT_embedding(test_input_ids, test_attention_masks)




## Ridge Regression

In [0]:
clf = Ridge(alpha=2)
clf.fit(train_features, train_labels)

predictions = clf.predict(test_features)

pearson = pearsonr(test_labels, predictions)
print(f'RMSE: {rmse(predictions, test_labels)} Pearson {pearson[0]}')
'RMSE: 0.861979277558717 Pearson 0.3621685325738159'

## SVR with different kernel

In [0]:
for k in ['linear','poly','rbf','sigmoid']:
    clf_t = SVR(kernel=k)
    clf_t.fit(train_features, train_labels)
    print(k)
    predictions = clf_t.predict(test_features)
    pearson = pearsonr(test_labels, predictions)
    print(f'RMSE: {rmse(predictions,test_labels)} Pearson {pearson[0]}')
    print()


'''linear
RMSE: 0.8847769222998597 Pearson 0.3541554106726864

poly
RMSE: 0.8800424227174211 Pearson 0.3812763230624585

rbf
RMSE: 0.8846797573743066 Pearson 0.3739785444174556

sigmoid
RMSE: 0.9047304999833146 Pearson 0.3336589772215751'''

# Results

(Haven't tested the function yet...)

In [0]:
import os
from google.colab import files
from zipfile import ZipFile

def writeScores(scores):
    fn = "predictions.txt"
    print("")
    with open(fn, 'w') as output_file:
        for idx,x in enumerate(scores):
            #out =  metrics[idx]+":"+str("{0:.2f}".format(x))+"\n"
            #print(out)
            output_file.write(f"{x}\n")


def downloadScores(method_name, scores):
  writeScores(scores)
  with ZipFile(f"en-zh_{method_name}.zip", "w") as newzip:
    newzip.write("predictions.txt")
  
  files.download(f"en-zh_{method_name}.zip")